In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm;
from pickle import dump, load

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torchsummary
import torch.backends.cudnn as cudnn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, plot_confusion_matrix

import albumentations
from albumentations.pytorch.transforms import ToTensorV2

import random
import librosa
from librosa import display

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

CLASS_NUM = 23
lr = 1e-4
batch_size = 512
epoch = 100
patience = 100

num_channels, num_rows, num_columns = 1, 40, 1292

c:\Users\cau\miniconda3\envs\het\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
seed = 42

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(seed)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, label, transform=None):
        self.data = data
        self.label = label
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, idx):
        data = self.data[idx]
        label = self.label[idx]
            
        return torch.tensor(data, dtype=torch.float), torch.tensor(label, dtype=torch.long)

In [ ]:
def get_accuracy(model, data_loader, device):
    running_correct = 0.0
    size = len(data_loader.dataset)
    with torch.no_grad():
        model.eval()
        for inputs, labels in data_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            # 답 데이터
            _, pred = torch.max(outputs, 1)
            running_correct += torch.sum(pred == labels)
    running_correct = running_correct.double() / size
    
    return running_correct

def plot_losses(train_losses, valid_losses):
    plt.style.use('seaborn')
    
    train_losses = np.array(train_losses)
    valid_losses = np.array(valid_losses)
    
    fig, ax = plt.subplots(figsize=(8, 4.5))
    
    ax.plot(train_losses, color='blue', label='Training loss')
    ax.plot(valid_losses, color='red', label='Validation loss')
    ax.set(title='Loss over epochs',
           xlabel='Epoch',
           ylabel='Loss')
    ax.legend()
    fig.show()
    
    plt.style.use('default')

def plot_accuracy(train_accuracy, valid_accuracy):
    plt.style.use('seaborn')
    
    train_accuracy = np.array(train_accuracy)
    valid_accuracy = np.array(valid_accuracy)
    
    fig, ax = plt.subplots(figsize=(8, 4.5))
    
    ax.plot(train_accuracy, color='blue', label='Training accuracy')
    ax.plot(valid_accuracy, color='red', label='Validation accuracy')
    ax.set(title='accuracy over epochs',
           xlabel='Epoch',
           ylabel='accuracy')
    ax.legend()
    fig.show()
    
    plt.style.use('default')

In [ ]:
def train(train_loader, model, criterion, optimizer, device):
    model.train()
    running_loss= 0.0
    epoch_loss = 0.0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()

        inputs = inputs.to(device)
        labels = labels.to(device)

        # prop
        outputs = model(inputs)
        print(outputs.shape)
        print(labels.shape)
        loss = criterion(outputs, labels)
        # backprop
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        
    epoch_loss = running_loss / len(train_loader.dataset)
    
    running_correct = get_accuracy(model, train_loader, device)
    
    return model, optimizer, epoch_loss, running_correct.cpu()

In [ ]:
def validate(valid_loader, model, criterion, device):
    model.eval()
    running_loss = 0.0
    epoch_loss = 0.0
    
    for inputs, labels in valid_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # prop and loss
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * inputs.size(0)
        
    epoch_loss = running_loss / len(valid_loader.dataset)
    
    running_correct = get_accuracy(model, valid_loader, device)
    
    return model, epoch_loss, running_correct.cpu()

In [ ]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience, verbose=False, delta=0, stop_active=False):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.stop_active = stop_active
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            
            if self.stop_active == True:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
                if self.counter >= self.patience:
                    self.early_stop = True
            else:
                print(f'EarlyStopping counter: {self.counter}')
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), './' + model.__class__.__name__ + '_best_model.pt')
        self.val_loss_min = val_loss

In [ ]:
from datetime import datetime

def training_loop(model, criterion, optimizer,
                  train_loader, valid_loader,
                  epochs, device, early_stopping, print_every=1):
    # best_loss = 1e10

    train_losses = []
    valid_losses = []
    train_acces = []
    valid_acces = []
    
    for epoch in range(0, epochs):
        # training
        model, optimizer, train_loss, train_acc = train(train_loader, model,
                                             criterion, optimizer, device)
        train_losses.append(train_loss)
        train_acces.append(train_acc)
        
        # validation
        with torch.no_grad():
            model, valid_loss, valid_acc = validate(valid_loader, model,
                                         criterion, device)
            valid_losses.append(valid_loss)
            valid_acces.append(valid_acc)
            
            early_stopping(valid_loss, model)
        
        if epoch % print_every == (print_every - 1):
            
            print(
                  f'{datetime.now().time().replace(microsecond=0)} ---'
                  f'Epoch: {epoch + 1}\t'
                  f'Train loss: {train_loss:.4f}\t'
                  f'Valid loss: {valid_loss:.4f}\t'
                  f'Train accuracy: {100 * train_acc:.2f}%\t\t'
                  f'Valid accuracy: {100 * valid_acc:.2f}%')
        
        if early_stopping.early_stop == True:
            break
    
    plot_losses(train_losses, valid_losses)
    plot_accuracy(train_acces, valid_acces)

    model_state_dict = torch.load('./' + model.__class__.__name__ + '_best_model.pt', map_location=device)
    model.load_state_dict(model_state_dict)
    test_acc = get_accuracy(model, test_loader, device)
    print(f'test accuracy: {100 * test_acc:.2f}%')

    return model, optimizer
    # , (train_losses, valid_losses)

In [ ]:
class Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(num_rows * num_columns, CLASS_NUM)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.linear(x)


class SVM(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(num_rows * num_columns, CLASS_NUM)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.linear(x)


class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.feature_extractor = nn.Sequential(            
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=(2, 2)),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=(2, 2)),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=16 * 10 * 323, out_features=120),
            nn.ReLU(),
            nn.Linear(in_features=120, out_features=84),
            nn.ReLU(),
            nn.Linear(in_features=84, out_features=CLASS_NUM)
        )
        
    def forward(self, x):
        x = x.view(x.size(0), num_channels, num_rows, num_columns)
        x = self.feature_extractor(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [ ]:
tag, scale = 'mfcc', 'test'
features_df = pd.read_json('../Processed_Data/data_' + tag + '_' + scale + '.json')
x = np.array(features_df.feature.tolist())
y = np.array(features_df.class_label.tolist())

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=seed)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=seed)

scaler = MinMaxScaler()
scaler.fit(x_train.reshape(x_train.shape[0], -1))
x_train = scaler.transform(x_train.reshape(x_train.shape[0], -1))
dump(scaler, open('../Saved_Scale/minmax_scaler.pkl', 'wb'))
x_val = scaler.transform(x_val.reshape(x_val.shape[0], -1))
x_test = scaler.transform(x_test.reshape(x_test.shape[0], -1))

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)

train_dataset = CustomDataset(data=x_train, label=y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = CustomDataset(data=x_val, label=y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataset = CustomDataset(data=x_test, label=y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
early_stopping = EarlyStopping(patience, verbose=True)

model = SVM().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.MultiMarginLoss()
torchsummary.summary(model, input_size=(1, 40, 1292))

In [ ]:
# vram 캐시를 없앱니다.
import gc
gc.collect()
torch.cuda.empty_cache()

model, optimizer = training_loop(model, criterion, optimizer,
                                    train_loader, val_loader,
                                    epoch, device, early_stopping)

In [ ]:
import torch
import torch.nn as nn

loss = nn.MultiMarginLoss()
x = torch.tensor([[0.1, 0.2, 0.4, 0.8],
                  [0.1, 0.2, 0.4, 0.8],
                  [0.1, 0.2, 0.4, 0.8],
                  [0.1, 0.2, 0.4, 0.8],
                  [0.1, 0.2, 0.4, 0.8]
                  ])
print(x.shape)
y = torch.tensor([3, 2, 1, 0, 1])
print(y.shape)
# 0.25 * ((1-(0.8-0.1)) + (1-(0.8-0.2)) + (1-(0.8-0.4)))
loss(x, y)